In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../smplh_rig')
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

import cv2
from utils.utils_display import DisplayHand
from utils.utils_mediapipe import MediaPipeHand

pipe = MediaPipeHand(static_image_mode=False, max_num_hands=2)
disp = DisplayHand(max_num_hands=2)

xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='SMPLH-Rig',rel_xml_path=xml_path,VERBOSE=False)
env.Hz = 30

for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [1,0.7,0.7,0]

env.init_viewer(viewer_title='Joint Pos',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=1.0,elevation=-21,lookat=[0.02,-0.03,0.8])
env.reset()

hand_limbs = [[0,1],[1,2],[2,3],[3,4],
              [0,5],[5,6],[6,7],[7,8],
              [5,9],[9,10],[10,11],[11,12],
              [9,13],[13,14],[14,15],[15,16],
              [0,17],[13,17],[17,18],[18,19],[19,20]]

MuJoCo version:[3.1.3]
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


I0000 00:00:1710405572.113005   47133 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710405572.200634   47434 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.161.07), renderer: NVIDIA GeForce RTX 3060/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
valid_cams = []
for i in range(8):
    cap = cv2.VideoCapture(i)
    if cap is None or not cap.isOpened():
        print('Warning: unable to open video source: ', i)
    else:
        valid_cams.append(i)

print(valid_cams)

[2, 4]


[ WARN:0@1.957] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1.959] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.959] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@1.961] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.963] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@1.964] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.966] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@1.968] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@1.968] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@1.969] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup C

In [3]:
cap = cv2.VideoCapture(4)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    img = cv2.flip(img, 1)
    img.flags.writeable = False
    param = pipe.forward(img)

    cv2.imshow('original_video', img)
    cv2.imshow('hands_3d', disp.draw2d_(img.copy(), param))
    pos0 = param[0]['joint'].copy()
    pos0 = pos0[:,[2,0,1]]
    pos0 *= np.array([-1,1,-1])
    pos0 += np.array([0,0,0.5])

    pos1 = param[1]['joint'].copy()
    pos1 = pos1[:,[2,0,1]]
    pos1 *= np.array([-1,1,-1])
    pos1 += np.array([0,0,0.5])

    for i in range(pos0.shape[0]):
        env.plot_T(p=pos0[i,:],R=np.eye(3),PLOT_AXIS=False,PLOT_SPHERE=True,sphere_r=0.005)

    for j in hand_limbs:
        env.plot_cylinder_fr2to(p_fr=pos0[j[0]],p_to=pos0[j[1]],r=0.003,rgba=[0.7,0.7,0.7,1])

    for i in range(pos0.shape[0]):
        env.plot_T(p=pos1[i,:],R=np.eye(3),PLOT_AXIS=False,PLOT_SPHERE=True,sphere_r=0.005)

    for j in hand_limbs:
        env.plot_cylinder_fr2to(p_fr=pos1[j[0]],p_to=pos1[j[1]],r=0.003,rgba=[0.7,0.7,0.7,1])

    env.render()

    key = cv2.waitKey(1)
    if key==27:
        break

cap.release()
env.close_viewer()